# 基于概率论的分类方法：——朴素贝叶斯
* 使用概率分布进行分类
* 学习朴素贝叶斯分类器
* 解析RSS源数据
* 使用朴素贝叶斯来分析不同地区的态度

朴素贝叶斯
* 优点： 在数据缺少的情况下依然有效
* 缺点： 对于输入数据的准备方式较为敏感
* 适用数据类型： 标称型数据

## 使用朴素贝叶斯进行文档分类

一般过程
* 收集数据： 任何方法，此处使用RSS源
* 准备数据： 数值型或bool型
* 分析数据： 大量特征，使用直方图
* 训练算法： 计算不同的独立特征的条件概率
* 测试算法： 计算错误率
* 使用算法： 常见应用，文档分类

特征条件独立性假设
* 特征之间相互独立
* 每个特征同等重要

朴素贝叶斯分类器实现方式（针对文档分类）
* 基于伯努力模型，只考虑文档中词是否出现
* 基于多项式模型实现，考虑词在文档中的出现次数

### 使用Python进行文本分类
#### 从文本中构建词向量

In [1]:
from numpy import *

输入数据样本

In [2]:
def loadDataSet():
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]        # 1 for insult, 0 for not
    return postingList, classVec

根据所有输入文本构建词集

In [3]:
def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

根据词集将每个输入文本转换为词向量

In [4]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print 'the word: %s is not in my Vocabulary!' % word
    return returnVec

In [5]:
listOPosts, listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)

In [6]:
myVocabList

['cute',
 'love',
 'help',
 'garbage',
 'quit',
 'I',
 'problems',
 'is',
 'park',
 'stop',
 'flea',
 'dalmation',
 'licks',
 'food',
 'not',
 'him',
 'buying',
 'posting',
 'has',
 'worthless',
 'ate',
 'to',
 'maybe',
 'please',
 'dog',
 'how',
 'stupid',
 'so',
 'take',
 'mr',
 'steak',
 'my']

In [7]:
setOfWords2Vec(myVocabList, listOPosts[0])

[0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1]

In [8]:
setOfWords2Vec(myVocabList, listOPosts[3])

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0]

#### 从词向量计算概率

* trainMatrix： 输入词向量矩阵（这里应该是列表的列表），长度等同于之前构造的词袋子，每个词对应一个特征
* trainCategory： 每个词向量对应的标签，侮辱性或者非侮辱性
* pAbusive： 侮辱性词向量数目
* p1Num： 矩阵，对应的值为统计侮辱性词向量中各个词的数目
* p1Denom： 标量，对应的为统计侮辱性词向量中的所有词数目，注意与pAbusive的区别

由上
* 先验概率（文档属于侮辱类的概率） $p(c_i)=\frac {pAbusive}{numTrainDocs}$
* 条件概率（在给定文档类别条件下词汇表中单词的出现概率） $p(c_i|w)=\frac {p1Num}{p1Denom}$

In [9]:
def trainNBO(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    # prior prob P(1)
    pAbusive = sum(trainCategory)/float(numTrainDocs)
#     p0Num = zeros(numWords)
#     p1Num = zeros(numWords)
    p0Num = ones(numWords)
    p1Num = ones(numWords)
#     p0Denom = 0.0
#     p1Denom = 0.0
    p0Denom = 2.0
    p1Denom = 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    # condition prob vector
#     p1Vect = p1Num / p1Denom
#     p0Vect = p0Num / p0Denom
    p1Vect = log(p1Num / p1Denom)
    p0Vect = log(p0Num / p0Denom)
    
    return p0Vect, p1Vect, pAbusive

In [10]:
listOPosts

[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]

In [11]:
trainMat = []
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))

In [12]:
for i in trainMat:
    print i

[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]


计算属于侮辱性文档的先验概率以及不同类别下各词（每个词是一个特征）的条件概率向量

In [13]:
p0V, p1V, pAb = trainNBO(trainMat, listClasses)

In [14]:
p0V

array([-2.56494936, -2.56494936, -2.56494936, -3.25809654, -3.25809654,
       -2.56494936, -2.56494936, -2.56494936, -3.25809654, -2.56494936,
       -2.56494936, -2.56494936, -2.56494936, -3.25809654, -3.25809654,
       -2.15948425, -3.25809654, -3.25809654, -2.56494936, -3.25809654,
       -2.56494936, -2.56494936, -3.25809654, -2.56494936, -2.56494936,
       -2.56494936, -3.25809654, -2.56494936, -3.25809654, -2.56494936,
       -2.56494936, -1.87180218])

#### 根据现实情况修改分类器
当前存在的一些问题
* 在计算$p(w|c)$时由于使用了条件独立性假设，所以是计算各个类别下某文档出现的概率的乘积，而一旦其中任一文档概率为0,则最后的成绩也为0，所以需要进行调整，可以将所有词出现数初始化为1，对应的分母初始化为2
* 下溢出，依然是条件概率的相乘带来的问题，太多的很小的数相乘会造成该问题，可以采用求对数的方法来避免下溢出或者浮点数舍入导致的错误

#### 朴素贝叶斯分类函数
* 由于前面进行了一些对数化处理，所以利用贝叶斯定理计算后验概率(不计算分母的联合概率)时原始的相乘操作均转化为相加操作，并在‘加’先验概率之前对其先进行对数操作


In [15]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    # element wise
    p1 = sum(vec2Classify*p1Vec) + log(pClass1)
    p0 = sum(vec2Classify*p0Vec) + log(1-pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

In [16]:
def testingNB():
    listOPosts, listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V, p1V, pAb = trainNBO(trainMat, listClasses)
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print testEntry, 'classified as: ',classifyNB(thisDoc, p0V, p1V, pAb)
    testEntry = ['stupid', 'garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print testEntry, 'classified as: ',classifyNB(thisDoc, p0V, p1V, pAb)

In [17]:
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


#### 文档词袋模型
* 前述为词集模型，这里构建词袋模型
* 两者区别在于，词集仅统计某词是否出现，而词袋则统计的是各词出现次数

In [35]:
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
#         else:
#             print 'the word: %s is not in my Vocabulary!' % word
    return returnVec

### 使用朴素贝叶斯过滤垃圾邮件
* 收集数据： 所提供的文本文件
* 准备数据： 将文本文件解析为词条向量
* 分析数据： 检查词条确保解析的正确性
* 训练算法： 使用前面构建的trainNBO()函数
* 测试算法： 使用classifyNB(),并构建一个新的测试函数来计算文档集的错误率
* 使用算法： 构建一个完整程序对一组文档进行分类，将错分的文档输出到屏幕上

#### 切分文本
将文本解析为词条

In [19]:
import re

In [20]:
def textParse(bigString):
    listOfTokens = re.split(r'\W*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok)>2]

#### 使用朴素贝叶斯进行交叉验证

In [21]:
def spamTest():
    docList = []
    classList = []
    fullText = []
    # load text file and parse it
    for i in range(1, 26):
        # spam
        wordList = textParse(open('email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        # ham
        wordList = textParse(open('email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)
    trainingSet = range(50)
    testSet = []
    # construct train/test set randomly
    # train/test Set only store the index of doc
    for i in range(10):
        randIndex = int(random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
    # construct word vector and train model with training matrix
    trainMat = []
    trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V, p1V, pSpam = trainNBO(array(trainMat), array(trainClasses))
    # test model and count error rate
    errorCount = 0
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])
        if classifyNB(array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
            print 'classification error ', docList[docIndex]
#     print 'the error rate is: ', float(errorCount) / len(testSet)
    error_rate = float(errorCount) / len(testSet)
    return error_rate

In [22]:
spamTest()

classification error  ['yeah', 'ready', 'may', 'not', 'here', 'because', 'jar', 'jar', 'has', 'plane', 'tickets', 'germany', 'for']


0.1

In [23]:
sum_error_rate = 0.0
for i in range(10):
    error_rate = spamTest()
    sum_error_rate += error_rate
    print 'error rate of epoch {0} is {1}'.format(i+1, error_rate)
print 'total error rate is: ', sum_error_rate/10

error rate of epoch 1 is 0.0
error rate of epoch 2 is 0.0
classification error  ['home', 'based', 'business', 'opportunity', 'knocking', 'your', 'door', 'don', 'rude', 'and', 'let', 'this', 'chance', 'you', 'can', 'earn', 'great', 'income', 'and', 'find', 'your', 'financial', 'life', 'transformed', 'learn', 'more', 'here', 'your', 'success', 'work', 'from', 'home', 'finder', 'experts']
error rate of epoch 3 is 0.1
classification error  ['yeah', 'ready', 'may', 'not', 'here', 'because', 'jar', 'jar', 'has', 'plane', 'tickets', 'germany', 'for']
error rate of epoch 4 is 0.1
error rate of epoch 5 is 0.0
error rate of epoch 6 is 0.0
classification error  ['home', 'based', 'business', 'opportunity', 'knocking', 'your', 'door', 'don', 'rude', 'and', 'let', 'this', 'chance', 'you', 'can', 'earn', 'great', 'income', 'and', 'find', 'your', 'financial', 'life', 'transformed', 'learn', 'more', 'here', 'your', 'success', 'work', 'from', 'home', 'finder', 'experts']
error rate of epoch 7 is 0.1
err

_注： 当前计算的错误率是针对将垃圾邮件误判为正常邮件时的错误，后续可以进一步改进_

### 使用朴素贝叶斯分类器从个人广告中获取区域倾向
解释朴素贝叶斯分类器训练所得到的知识
* 从美国的两个城市中选取一些人，通过分析这些人发布的征婚广告信息，来比较这两个城市的人们在广告用词上是否不同
* 若结论确实不同，判断各自常用词，从而对不同城市的人所关心的内容有所了解

* 收集数据： 从RSS源收集内容，需要对RSS源构建一个接口
* 准备数据： 将文本文件解析成词条向量
* 分析数据： 检查词条确保解析的正确性
* 训练算法： 使用之前构建的trainNBO()函数
* 测试算法： 观察错误率，确保分类器可用，可修改切分程序
* 使用算法： 构建一个完整程序，封装所有内容，给定两个RSS源，该程序会显示最常用的公共词

#### 导入RSS源

In [24]:
import feedparser

In [25]:
ny = feedparser.parse('http://newyork.craigslist.org/stp/index.rss')

In [26]:
ny['entries']

[{u'dc_source': u'http://newyork.craigslist.org/mnh/stp/6155532416.html',
  u'dc_type': u'text',
  u'id': u'http://newyork.craigslist.org/mnh/stp/6155532416.html',
  u'language': u'en-us',
  u'link': u'http://newyork.craigslist.org/mnh/stp/6155532416.html',
  u'links': [{u'href': u'http://newyork.craigslist.org/mnh/stp/6155532416.html',
    u'rel': u'alternate',
    u'type': u'text/html'}],
  u'published': u'2017-06-12T03:52:12-04:00',
  u'published_parsed': time.struct_time(tm_year=2017, tm_mon=6, tm_mday=12, tm_hour=7, tm_min=52, tm_sec=12, tm_wday=0, tm_yday=163, tm_isdst=0),
  u'rights': u'copyright 2017 craiglist',
  u'rights_detail': {u'base': u'https://newyork.craigslist.org/search/stp?format=rss',
   u'language': None,
   u'type': u'text/plain',
   u'value': u'copyright 2017 craiglist'},
  u'summary': u"Seeking a true DAD/son relationship with a chill masculine blk brother \nHi first upfont I have to tell you that I' deal with females \nso if that is an issue for you then there

In [27]:
len(ny['entries'])

25

去除高频词

In [28]:
import operator

In [29]:
def calcMostFreq(vocabList, fullText):
    freqDict = {}
    for token in vocabList:
        freqDict[token] = fullText.count(token)
    sortedFreq = sorted(freqDict.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortedFreq[:30]

测试函数

In [37]:
def localWords(feed1, feed0):
    docList = []
    classList = []
    fullText = []
    minLen = min(len(feed1['entries']), len(feed0['entries']))
    # parse data from RSS
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    # remove high freq words
    vocabList = createVocabList(docList)
    top30Words = calcMostFreq(vocabList, fullText)
    for pairW in top30Words:
        if pairW[0] in vocabList:
            vocabList.remove(pairW[0])
    # construct train/test set randomly
    trainingSet = range(2*minLen)
    testSet = []
    for i in range(20):
        randIndex = int(random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
    # construct train matrix
    trainMat = []
    trainClasses = []    
    for docIndex in trainingSet:
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    # train model to count the prior/condition prob
    p0V, p1V, pSpam = trainNBO(array(trainMat), array(trainClasses))
    # test model
    errorCount = 0
    for docIndex in testSet:
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
    print 'the error rate is: ', float(errorCount) / len(testSet)
    
    return vocabList, p0V, p1V

In [31]:
ny = feedparser.parse('http://newyork.craigslist.org/stp/index.rss')
sf = feedparser.parse('http://sfbay.craigslist.org/stp/index.rss')

In [38]:
vocabList, pSF, pNY = localWords(ny, sf)

the error rate is:  0.25


#### 显示地域相关的用词
先对向量pSF和pNY进行排序，然后按照顺序将词打印出来

In [41]:
def getTopWords(ny, sf):
    vocabList, p0V, p1V = localWords(ny, sf)
    topNY = []
    topSF = []
    for i in range(len(p0V)):
        if p0V[i] > -6.0:
            topSF.append((vocabList[i], p0V[i]))
        if p1V[i] > -6.0:
            topNY.append((vocabList[i], p1V[i]))
    sortedSF = sorted(topSF, key=lambda pair: pair[1], reverse=True)
    print '********************SF***********************************'
    for item in sortedSF:
        print item[0]
    sortedNY = sorted(topNY, key=lambda pair: pair[1], reverse=True)
    print '********************NY***********************************'
    for item in sortedNY:
        print item[0]

In [46]:
getTopWords(ny, sf)

the error rate is:  0.35
********************SF***********************************
them
are
guy
massage
maybe
really
feel
also
wear
area
vers
very
here
strrings
live
music
relax
movies
male
sit
travel
stress
years
good
platonic
believe
than
thhongs
play
tall
professional
bay
wears
likes
has
well
lady
old
ladies
lingerrie
oral
augustine
conversation
young
blaze
manis
every
join
school
having
tissue
enjoy
cost
traveling
uncomfortable
blue
what
abo
new
men
explore
let
meet
alone
wait
great
suggests
smoke
snobby
lounges
use
from
positive
bdsm
today
females
rhythm
hole
women
practicing
room
athlete
something
give
way
swedish
beach
petite
may
such
guys
man
talk
mainly
attitude
through
looks
style
thank
actually
pure
safe
mention
they
now
day
bigger
name
lingerie
friendship
idea
girl
special
since
attracted
dominated
lag
california
recovering
could
days
think
already
soothing
feet
city
toy
jet
rates
eyes
fraction
hotel
park
king
grew
favors
individual
san
need
any
sensual
invite
sell
10084
ab